In [1]:
import os
os.environ["PYTHON_EXEC"] = "/Users/jessieyu/.pyenv/versions/qka-demo/bin/python3"
os.environ["NTC_DOC_FILE"] = "runtime/program_doc.json"
os.environ["NTC_PROGRAM_FILE"] = "runtime/qka_runtime_outer.py"

# List available quantum programs

In [2]:
from qiskit import IBMQ

provider = IBMQ.load_account()
provider.runtime.programs()

"Quantum-Kernel-Alignment":
  Description: Quantum kernel alignment algorithm that learns, on a given dataset, a quantum kernel maximizing the SVM classification margin.
  Parameters:
    - feature_map:
      description: An instance of FeatureMapForrelation in dictionary format used to map classical data into a quantum state space.
      type: dict
    - data:
      description: NxD array of training data, where N is the number of samples and D is the feature dimension.
      type: numpy.ndarray
    - labels:
      description: Nx1 array of +/-1 labels of the N training samples.
      type: numpy.ndarray
    - lambda_initial:
      description: Initial parameters of the quantum feature map.
      type: numpy.ndarray
    - spsa_steps:
      description: Number of SPSA optimization steps.
      type: int
    - C:
      description: Penalty parameter for the soft-margin support vector machine.
      type: float
  Returns:
    - aligned_kernel_parameters:
      description: The optimized 

# Invoke Quantum Kernel Alignment program

In [3]:
import numpy as np
from qka.featuremaps import FeatureMapForrelation

num_samples=5  # number of samples per class in the input data
num_features=2 # number of features in the input data
depth=2        # depth of the feature map circuit
C=1            # SVM soft-margin penalty
spsa_steps=2   # number of SPSA iterations

# Define the feature map and its initial parameters:
fm = FeatureMapForrelation(feature_dimension=num_features, depth=depth, entangler_map=None)
lambda_initial = np.random.uniform(-1,1, size=(fm._num_parameters))

# create random test data and labels:
x_train = np.random.rand(2*num_samples, num_features)
y_train = np.concatenate((-1*np.ones(num_samples), np.ones(num_samples)))

backend = provider.backend.ibmq_qasm_simulator

In [4]:
def interim_result_callback(interim_result):
    print(f"interim result: {interim_result}\n")

In [5]:
# Call QKA runtime program.
runtime_params = {
    'feature_map': fm.to_dict(),
    'data': x_train,
    'labels': y_train,
    'lambda_initial': lambda_initial,
    'spsa_steps': 2,
    'C': C
}
options = {'backend_name': backend.name()}
result = provider.runtime.run(program_name="Quantum-Kernel-Alignment",
                              options=options,
                              params=runtime_params,
                              callback=interim_result_callback,
                             ).result()
print("final result:")
print(f"aligned_kernel_parameters: {result['aligned_kernel_parameters']}")
print(f"aligned_kernel_matrix: {result['aligned_kernel_matrix']}")

interim result: {'cost': 8.366700209755901, 'lambda': array([-0.43848082,  0.33305944, -0.36154461,  0.47137867]), 'cost_plus': 7.978517695802974, 'cost_minus': 8.754882723708826, 'cost_final': 8.366700209755901}

interim result: {'cost': 7.304830508235804, 'lambda': array([-0.45233931,  0.31920095, -0.34768613,  0.48523715]), 'cost_plus': 7.500955023334846, 'cost_minus': 7.108705993136761, 'cost_final': 7.304830508235804}

final result:
aligned_kernel_parameters: [-0.08908201  0.06522604 -0.07092307  0.09566158]
aligned_kernel_matrix: [[1.         0.73925781 0.3046875  0.25683594 0.55566406 0.33398438
  0.86035156 0.00976562 0.70898438 0.31152344]
 [0.73925781 1.         0.77246094 0.74804688 0.94238281 0.60351562
  0.97265625 0.1328125  0.93847656 0.42285156]
 [0.3046875  0.77246094 1.         0.94824219 0.91601562 0.75585938
  0.66796875 0.28613281 0.77734375 0.47070312]
 [0.25683594 0.74804688 0.94824219 1.         0.87304688 0.59179688
  0.59667969 0.30273438 0.6953125  0.35351562

# Test the aligned kernel

In [6]:
from qka.kernel_matrix import KernelMatrix
from sklearn.svm import SVC
from sklearn import metrics

# Test the aligned kernel on test data:

x_test = np.random.rand(2*num_samples, num_features)
y_test = np.concatenate((-1*np.ones(num_samples), np.ones(num_samples)))

kernel_aligned = result['aligned_kernel_matrix']
model = SVC(C=C, kernel='precomputed')
model.fit(X=kernel_aligned, y=y_train)

km = KernelMatrix(feature_map=fm, backend=backend)
kernel_test = km.construct_kernel_matrix(x1_vec=x_test, x2_vec=x_train, parameters=result['aligned_kernel_parameters'])
labels_test = model.predict(X=kernel_test)
accuracy_test = metrics.balanced_accuracy_score(y_true=y_test, y_pred=labels_test)
print(f"accuracy test: {accuracy_test}")

accuracy test: 0.6000000000000001
